gemaakt voor LIWO door [David Haasnoot](d.haasnoot@hkv.nl)

In [ ]:
import pandas as pd
import dotenv

In [ ]:
from update_local_bulk_LDO import haal_scenarios_op, haal_token_op

Zie `readme.md` voor uitgebreid stappen plan voor het aanmaken van een api key.

In het kort
- Login & ga naar https://www.overstromingsinformatie.nl/auth/
- Onder `V1`, `POST auth/v1/personalapikeys`, klik op `try it out` en vervang body met:
```json
{
  "scope": "*:readwrite",
  "name": "personalAPIkeyLIWOexport...",
  "expiry_date": "2029-12-31T23:59:59.037Z",
  "revoked": false
}
```
- Pas de datum en name aan waar nodig, klik op `Excecute` in het blauw. In de response body staat: 
```json
{
...
  "key": "xxxxxx.yyyyyyyyyyyyyyy",
...
}
```
- Bewaar die hele `'key'` in een bestand die `.env` heet, zie `.env.example` voor het formaat.
- Afhankelijk via welke organisatie je toegang hebt, kan het nodig zijn om in de code de `TENANT` variabele aan te passen. Deze kan ook in de `.env` file worden gezet.
  - `TENANT = 0` voor beheerders? 
  - `TENANT = 1` voor LIWO
  - `TENANT = 2` voor RWS
  ...

In [ ]:
# haal de API key op uit de .env file
if dotenv.load_dotenv():
    environmental_variables = dotenv.dotenv_values()
    LDO_api_key = environmental_variables["LDO_api_key"]\
# Of zet hier de API key handmatig in
# LDO_api_key = "abcd"
TENANT : int  = 1 # 0, 1, 2 ...
headers = haal_token_op(LDO_api_key, tenant=TENANT)

In [ ]:
maximum = 10_000
beschikbare_scenario_ids = haal_scenarios_op(maximum, headers)

In [ ]:
len(beschikbare_scenario_ids)

In [ ]:
from export_LDO import get_ssm

In [ ]:
lst_json = [get_ssm(scenario, headers) for scenario in beschikbare_scenario_ids]


In [ ]:
df_metadata = pd.DataFrame(lst_json)
df_metadata.set_index('scenario_id', inplace=True)
df_metadata.sort_index(inplace=True)

In [ ]:
all_indexes = df_metadata.index

In [ ]:
filled_scenarios = df_metadata.dropna(subset=['raster_types'])

In [ ]:
len(set(all_indexes).difference(filled_scenarios.index))

In [ ]:
df_metadata.to_excel('metadata_ssm.xlsx')
filled_scenarios.to_excel('metadata_ssm_filled.xlsx')